In [1]:
from bac.simulate.namd import (Simulation, PME, BondConstraint, AtomConstraint, System,
                               HarmonicConstraint, VelocityRescaling, VelocityReassignment,
                               LangevinDynamics, BerendsenPressureCoupling, VerletIntegrator, 
                               Interaction)

# Minimization steps.
Minimization of the system is composed of 11 steps, with increseangly released constraints.

In [2]:
integrator = VerletIntegrator(timestep=2.0, first_step=100)

minimize = Simulation(integrator=integrator, number_of_steps=100, temperature=300)

In [3]:
minimize.minimization = True

minimize.parameter_type_CHARMM = False
minimize.amber = True
minimize.parameters = 'build/complex.top'
minimize.coordinates = 'build/complex.pdb'
minimize.read_exclusions = False

minimize.binary_output = False
minimize.name = 'minimize_0'
minimize.output_name = 'minimize_0'
minimize.output_energies = 500
minimize.output_pressure = 500

In [4]:
s = System()
s.excluded_interactions = Interaction.scaled_one_four
s.one_four_scaling = 0.833333
s.cutoff = 12
s.switching = True
s.switching_distance = 10
s.pairlist_distance = 13.5

s.non_bonded_frequency = 1
s.full_elect_frequency = 2
s.steps_per_cycle = 10

s.pme = PME(grid_spacing=1.0)

s.wrap_water = s.wrap_all = True
s.cell_basis_vector_1 = 72.449, 0.000, 0.000
s.cell_basis_vector_2 =  0.000, 93.247, 0.000
s.cell_basis_vector_3 =  0.000, 0.000, 79.185
s.cell_origin = -0.248, -0.156, -0.047

minimize.system = s

In [5]:
minimize.constraints.bond_constraint = BondConstraint(bonds='all', tolerance=0.00001, iterations=100)

harm_const = HarmonicConstraint(exponent=2, reference_position_file='build/complex.pdb')
harm_const.force_constant_file = 'constraint/f4.pdb'
harm_const.force_constant_column = 'B'
harm_const.scaling = 10
minimize.constraints.harmonic_constraint = harm_const

In [6]:
minimizations = [minimize]

for scaling, index, n_steps in zip([10*0.5**index for index in range(1, 10)] + [0], range(1, 11), [100]*9+[5000]):
    minim = next(minimizations[-1])
    minim.name = f'minimize_{index}'
    minim.output_name = f'minimize_{index}'
    minim.constraints.harmonic_constraint.scaling = scaling
    minim.number_of_steps = n_steps
    minimizations.append(minim)

Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.


In [7]:
print(minimize.encode())

timestep                  = 2.0
firsttimestep             = 100

minimization              = on
temperature               = 300.0
coordinates               = build/complex.pdb
parmfile                  = build/complex.top
outputname                = minimize_0
binaryoutput              = no
dcdunitcell               = yes
outputEnergies            = 500
outputPressure            = 500
amber                     = yes
readexclusions            = no
scnb                      = 2.0
paraTypeXplor             = on
paraTypeCharmm            = off
gromacs                   = off
cutoff                    = 12.0
switching                 = on
switchdist                = 10.0
vdwForceSwitching         = off
exclude                   = scaled1-4
1-4scaling                = 0.833333
dielectric                = 1.0
nonbondedScaling          = 1.0
vdwGeometricSigma         = False
limitdist                 = 0.0
LJcorrection              = False
pairlistdist              = 13.5
stepspercycle        

# Equilibration NVT

In [8]:
equilibrate_1 = next(minimize)
equilibrate_1.name = 'eq1'
equilibrate_1.output_name = 'eq1'
equilibrate_1.minimization = False

# 5000 -> 1000
equilibrate_1.number_of_steps = 1000

equilibrate_1.temperature_controller = VelocityReassignment(frequency=100, increment=1, hold_at=300)

equilibrate_1.restart_frequency = 50000
equilibrate_1.system.xst_frequency = 5000
equilibrate_1.dcd_frequency = 5000

# Equilibration NPT

In [9]:
equilibrate_2 = next(equilibrate_1)
equilibrate_2.name = 'eq2_0'
equilibrate_2.output_name = 'eq2_0'
equilibrate_2.timestep = 1.0
equilibrate_2.constraints.bond_constraint.bonds = 'water'
equilibrate_2.temperature_controller = LangevinDynamics(temperature=300, damping=5, applies_to_hydrogen=False)
equilibrate_2.pressure_controller = BerendsenPressureCoupling(target=1, compressibility=4.57e-5, relaxation_time=100, 
                                                              frequency=2, use_group_pressure=True)

equilibrate_2.restart_frequency = 5000

Deleting cell vectors.


/Users/kristofarkas/Developer/supproperty/supproperty/supproperty.py:113: UserWarning: 'BerendsenPressureCoupling' object has no attribute 'simulation'
  warnings.warn(f'{e}')


In [10]:
equilibrations = [equilibrate_2]

for scaling, x in zip([10*0.5**x for x in range(1, 10)] + [0], range(1, 11)):
    eq = next(equilibrations[-1])
    eq.name = f'eq2_{x}'
    eq.output_name = f'eq2_{x}'
    eq.constraints.harmonic_constraint.scaling = scaling
    equilibrations.append(eq)

Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.
Deleting cell vectors.


# Production

In [11]:
production = next(equilibrations[-1])
production.name = 'production'
production.output_name = 'production'
production.timestep = 2.0
production.constraints.bond_constraint.bonds = 'all'
production.constraints.harmonic_constraint = None
production.restart_frequency = 200000

Deleting cell vectors.


# Workflow

This is where Radical could fit in nicely

In [12]:
from bac.simulate import Workflow, Replica

In [13]:
wf = Workflow(resource='bw', r_dir='esmacs_bw')

In [14]:
wf.simulations = minimizations + [equilibrate_1] + equilibrations + [production]

In [15]:
wf.ensembles = [Replica(number_of=5)]

In [16]:
# This command created the input files and puts them in a folder called esmacs_bw.
wf.preprocess_simulations()

In [17]:
minimize.parameters

PosixPath('build/complex.top')